## AI Assistant Development and Experimentation Notebook

### Tools used: Pushover, OpenAI, Gradio, Python

In [ ]:
# import

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
import json
import os
import requests

In [ ]:
# secrets .env

load_dotenv(override=True)

openai = OpenAI()